### Import libraries & Load data

In [ ]:
!pip install opencv-python
!pip install mediapipe
!pip install moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import mediapipe as mp
import moviepy.editor as m_editor
import csv
import glob
from os import path

In [ ]:
input_dir = "../data/chr/"
output_dir = "../data/chr-landmarks/"

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
PL = mp_holistic.PoseLandmark
HL = mp_holistic.HandLandmark
mp_face_mesh = mp.solutions.face_mesh
holistic = mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

### Define Functions

In [ ]:
# clip input(.mp4) video to output(.mp4) video
# start_time and end_time in seconds
def extract_subclip(input, output, start_time, end_time):
  original_video = m_editor.VideoFileClip(input)
  cut_video = original_video.subclip(start_time, end_time)
  cut_video.write_videofile(output, codec="libx264")

### Extract coordinates

In [ ]:
csv_header = ['width', 
              'height',
              'fps',
              'frame',
              'PL-LEFT_WRIST',
              'PL-RIGHT_WRIST',
              'PL-LEFT_ELBOW',
              'PL-RIGHT_ELBOW',
              'PL-LEFT_SHOULDER',
              'PL-RIGHT_SHOULDER',
              'PL-LEFT_ANKLE',
              'PL-RIGHT_ANKLE',
              'PL-LEFT_HIP',
              'PL-RIGHT_HIP',
              'PL-NOSE',
              'LH-INDEX_FINGER_TIP',
              'RH-INDEX_FINGER_TIP',
              'LH-THUMB_TIP',
              'RH-THUMB_TIP',
              'LH-MIDDLE_FINGER_TIP',
              'RH-MIDDLE_FINGER_TIP',
              'LH-MIDDLE_FINGER_MCP',
              'RH-MIDDLE_FINGER_MCP',
              'LH-RING_FINGER_TIP',
              'RH-RING_FINGER_TIP',
              'LH-RING_FINGER_MCP',
              'RH-RING_FINGER_MCP',
              'LH-PINKY_TIP',
              'RH-PINKY_TIP',
              'LH-PINKY_MCP',
              'RH-PINKY_MCP',
              'LH-WRIST',
              'RH-WRIST',
              ]

In [ ]:
pattern = input_dir+"*.mp4"
videos = [path.basename(x) for x in glob.glob(pattern)]
print(videos) 


['couch.mp4', 'interview.mp4']


In [ ]:
for video in videos:
  filename = video.split('.')[0]

  # define output files
  output_csv = output_dir+filename+'-landmarks.csv'
  video_cut = output_dir+filename+'-cut.mp4'
  video_out = output_dir+filename+'-out.mp4'

  # extract subclip of the video
  extract_subclip(input_dir+video, video_cut, 0, 60)
  # load the video
  cap = cv2.VideoCapture(video_cut)

  # Get the video dimensions and FPS(input)
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))

  # Initialize the VideoWriter(output)
  #fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # You can also use "XVID" or "MJPG" for AVI files
  #out = cv2.VideoWriter(video_out, fourcc, fps, (width, height))

  with open(output_csv, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(csv_header)

    # Process the video frames
    frame_number = 0

    while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
        break
      
      # Convert the frame to RGB
      frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      # Process the frame with MediaPipe's Holistic module
      results = holistic.process(frame_rgb)

      # Draw holistic landmarks on the frame
      if results.pose_landmarks:
        current = results.pose_landmarks
        current_lh = results.left_hand_landmarks
        current_rh = results.right_hand_landmarks

        writer.writerow([width, 
              height,
              fps,
              frame_number,
              current.landmark[PL.LEFT_WRIST],
              current.landmark[PL.RIGHT_WRIST],
              current.landmark[PL.LEFT_ELBOW],
              current.landmark[PL.RIGHT_ELBOW],
              current.landmark[PL.LEFT_SHOULDER],
              current.landmark[PL.RIGHT_SHOULDER],
              current.landmark[PL.LEFT_ANKLE],
              current.landmark[PL.RIGHT_ANKLE],
              current.landmark[PL.LEFT_HIP],
              current.landmark[PL.RIGHT_HIP],
              current.landmark[PL.NOSE],
              current_lh.landmark[HL.INDEX_FINGER_TIP] if current_lh else None,
              current_rh.landmark[HL.INDEX_FINGER_TIP] if current_rh else None,
              current_lh.landmark[HL.THUMB_TIP] if current_lh else None,
              current_rh.landmark[HL.THUMB_TIP] if current_rh else None,
              current_lh.landmark[HL.MIDDLE_FINGER_TIP] if current_lh else None,
              current_rh.landmark[HL.MIDDLE_FINGER_TIP] if current_rh else None,
              current_lh.landmark[HL.MIDDLE_FINGER_MCP] if current_lh else None,
              current_rh.landmark[HL.MIDDLE_FINGER_MCP] if current_rh else None,
              current_lh.landmark[HL.RING_FINGER_TIP] if current_lh else None,
              current_rh.landmark[HL.RING_FINGER_TIP] if current_rh else None,
              current_lh.landmark[HL.RING_FINGER_MCP] if current_lh else None,
              current_rh.landmark[HL.RING_FINGER_MCP] if current_rh else None,
              current_lh.landmark[HL.PINKY_TIP] if current_lh else None,
              current_rh.landmark[HL.PINKY_TIP] if current_rh else None,
              current_lh.landmark[HL.PINKY_MCP] if current_lh else None,
              current_rh.landmark[HL.PINKY_MCP] if current_rh else None,
              current_lh.landmark[HL.WRIST] if current_lh else None,
              current_rh.landmark[HL.WRIST] if current_rh else None,
              ]
                )
                
      frame_number += 1

    cap.release()

Moviepy - Building video /content/drive/MyDrive/train-landmarks/couch-cut.mp4.
MoviePy - Writing audio in couch-cutTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/train-landmarks/couch-cut.mp4



t:  13%|█▎        | 188/1500 [00:07<01:36, 13.59it/s, now=None]

KeyboardInterrupt: ignored